In [5]:
#Install all the depencencies
!pip install langgraph langchain_community langchain_groq
import getpass, os
os.environ["GROQ_API_KEY"] = getpass.getpass(f"Please provide your GROQ KEY")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 5.6 MB/s eta 0:00:00
Please provide your GROQ KEY··········


In [32]:
#Importing all the libs
from typing import Literal, TypedDict
from langchain_groq import ChatGroq
from langgraph.graph import MessagesState, END
from langgraph.types import Command
from langchain_core.tools import tool as tl
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent

#Declaring the tool agents that will be in the sistem
Class_Types = ["MathTeacher"]
options = Class_Types + ["FINISH"]

#Here is the prompt that is passed to the supervisor, so he know how to make the next decision based on his task
system_prompt = (
    "You are a helpfull supervisor, your task is to manage the question between the"
    f" following teachers: {Class_Types}. Given the user request,"
    " respond teacher that should respond. When finished,"
    " respond with FINISH."
)

#Here we declare the route that will be used to pass the tasks between the agents
class Router(TypedDict):
    """Teacher to route to next. If no Teacher needed, route to FINISH."""

    next: Literal[*options]

#Here we declare the model that gonna be used in both of the agents
supervisorLLM = ChatGroq(model="llama-3.1-8b-instant")

#This is the state used in the route, to point how the process is going(And if need to be finish)
class State(MessagesState):
    next: str

#Here we are declaring the supervisor node, then he's able to make route control
def supervisor_node(state: State) -> Command[Literal[*Class_Types, "__end__"]]:
    messages = [
        {"role": "system", "content": system_prompt},
    ] + state["messages"]
    response = supervisorLLM.with_structured_output(Router).invoke(messages)
    step = response["next"]
    if step == "FINISH":
        step = END

    return Command(goto=step, update={"next": step})

In [33]:
#Next step is declare the "MathTeacher" Agent, starting by his system_prompt
teacher_prompt="You are a math teacher. Given one question or calculation, respond with the result."

#Then we declare the tools that we gonna use
@tl
def Subtract (a:int, b: int):
   """Subtract one value on another."""
   return a * b
@tl
def Add(a:int, b: int):
   """Add two numbers."""
   return a + b
@tl
def Divide(a:int, b: int):
   """Divide on number on another."""
   return a / b
@tl
def Multiply(a:int, b: int):
   """Multiply two numbers."""
   return a * b
@tl
def Exponentiate(a:int, b: int):
   """Exponentiate one number to the power of another."""
   return a ** b
#Integrating the Model, Tools and prompt, adding an state and command control, we got the agent that we need
MathTeacher_agent = create_react_agent(supervisorLLM,tools=[Subtract,Add,Divide,Multiply,Exponentiate],prompt=teacher_prompt)
def MathTeacher_node(state: State) -> Command[Literal["supervisor"]]:
    Qresult = MathTeacher_agent.invoke(state)
    humanMessage = HumanMessage(content=Qresult["messages"][-1].content, name="MathTeacher")
    return Command(update={"messages": [humanMessage]},goto="supervisor",)

#Finally we declare the graph, and push the nodes that integrates the system
builder = StateGraph(State)
builder.add_edge(START, "supervisor")
builder.add_node("supervisor", supervisor_node)
builder.add_node("MathTeacher", MathTeacher_node)
graph = builder.compile()

In [34]:
for s in graph.stream(
    {"messages": [("user", "How much is 47^5")]}, subgraphs=True):
    print(s)
    print("----")

((), {'supervisor': {'next': 'MathTeacher'}})
----
(('MathTeacher:29fa7721-59f4-c48e-ba73-d939fb65255d',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8f3e', 'function': {'arguments': '{"a": 47, "b": 5}', 'name': 'Exponentiate'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 474, 'total_tokens': 495, 'completion_time': 0.028, 'prompt_time': 0.018290745, 'queue_time': 0.241739814, 'total_time': 0.046290745}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ddb1cf135', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-930ad120-a780-482a-b947-b5b00b8667c1-0', tool_calls=[{'name': 'Exponentiate', 'args': {'a': 47, 'b': 5}, 'id': 'call_8f3e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 474, 'output_tokens': 21, 'total_tokens': 495})]}})
----
(('MathTeacher:29fa7721-59f4-c48e-ba73-d939fb65255d',), {'tools': {'messages': [ToolMessage(content='229345007', na